In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.append(r"D:/Clark/LFL_QubitLibraries")

# Introduction

In this tutorial, I show you how to use `QSweeper` to go through a parameter space given by `QComponent.options`, and then determine the best suited options for you.

<a href='#QSweeper'>Tutorial begins here</a>

### Make Design

In [3]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict

design = designs.DesignPlanar()
gui = MetalGUI(design)

design.overwrite_enabled = True

In [4]:
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

xmon = TransmonCross(design, 'xmon')
gui.rebuild()
gui.zoom_on_components(['xmon'])

  arr = construct_1d_object_array_from_listlike(values)

  arr = construct_1d_object_array_from_listlike(values)



In [5]:
xmon.options

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {},
 'cross_width': '20um',
 'cross_length': '200um',
 'cross_gap': '20um',
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

### Setting up `QAnalysis`

In [6]:
from QubitPresimulated.sweeper import QSweeper
from qiskit_metal.analyses.quantization import EPRanalysis

First initalize your analysis module. In this case, we'll be using `EPRanalysis`.

In [7]:
# Set up your EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

eig_qb.sim.setup.max_passes = 6
eig_qb.sim.setup.reuse_selected_design = False # MUST BE DEFINED AS SUCH ALWAYS
eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': False,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 1,
 'max_delta_f': 0.5,
 'max_passes': 6,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '10 nH', 'Cj': '0 fF'}}

In [8]:
eig_qb.setup.junctions.jj.rect = 'JJ_rect_Lj_xmon_rect_jj'
eig_qb.setup.junctions.jj.line = 'JJ_Lj_xmon_rect_jj_'

eig_qb.setup

{'junctions': {'jj': {'Lj_variable': 'Lj',
   'Cj_variable': 'Cj',
   'rect': 'JJ_rect_Lj_xmon_rect_jj',
   'line': 'JJ_Lj_xmon_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj'}

In [9]:
hfss = eig_qb.sim.renderer
hfss.start()
hfss.activate_ansys_design("TransmonQubit", 'eigenmode')  # use new_ansys_design() to force creation of a blank design
hfss.clean_active_design()
hfss.render_design(['xmon'], [])


INFO 04:22PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:22PM [load_ansys_project]: 	Opened Ansys App
INFO 04:22PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 04:22PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/lfl/Documents/Ansoft/
	Project:   Project64
INFO 04:22PM [connect_design]: 	Opened active design
	Design:    Design_hfss28 [Solution type: Eigenmode]
INFO 04:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:22PM [connect]: 	Connected to project "Project64" and design "Design_hfss28" 😀 

INFO 04:22PM [connect_design]: 	Opened active design
	Design:    TransmonQubit [Solution type: Eigenmode]


<a id='QSweeper'></a>
# Initalize `QSweeper`, and Run a Sweep

In [10]:
# It has one initalization input, which is a QAnalysis module
# In this case: type(eig_qb) is EPRanalysis
sweeper = QSweeper(eig_qb)

In [11]:
# Tell the program which component you want to sweep through
target_component = 'xmon'

# and which parameter to change. The format of this dict is the same as
# QComponent.options. So if you want to change claw_length:
# parameters_to_sweep = {'connection_pads':{'res_claw':{'claw_length':[1,2,3]}}}

# Basically, instead of having strings / floats as values
# you now have lists of values you want to sweep over.
parameters_to_sweep = {'cross_width': ['10um', '30um'],
                      'cross_length': ['150um','350um']}

# Anything after save_path is put in **kwargs which gets sent to
# analysis.run(**kwargs)
sweeper.run_sweep(component_name = target_component,
                  parameters = parameters_to_sweep,
                  data_name = None,
                  save_path = r'./QSweeper_Tutorial.csv',
                  components=['xmon']) # REQUIRED: must declare components= for this to wor


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]WARNING:py.warnings:D:\Anaconda\envs\metal_test_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)

INFO 04:22PM [connect_design]: 	Opened active design
	Design:    Design_hfss29 [Solution type: Eigenmode]
WARNING 04:22PM [connect_setup]: 	No design setup detected.
WARNING 04:22PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:22PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:22PM [analyze]: Analyzing setup Setup
04:22PM 55s INFO [get_f_convergence]: Saved convergences to D:\Clark\LFL_QubitLibraries\testing\hfss_eig_f_convergence.csv


Design "Design_hfss29" info:
	# eigenmodes    1
	# variations    1
Design "Design_hfss29" info:
	# eigenmodes    1
	# variations    1


  options=pd.Series(get_instance_vars(self.options)),




        energy_elec_all       = 8.21187825603725e-25
        energy_elec_substrate = 7.53621285742723e-25
        EPR of substrate = 91.8%

        energy_mag    = 8.05515568554797e-27
        energy_mag % of energy_elec_all  = 1.0%
        

Variation 0  [1/1]


  Ljs = pd.Series({})

  Cjs = pd.Series({})

  _Om = pd.Series({})



...

#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

,,0
Lj,,
10,0,76.28


  self.qoptions_data = self.qoptions_data.append(dict(zip(keys, values)), ignore_index=True)

  self.simulations_data = self.simulations_data.append(dict(zip(keys, values)), ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [02:27<00:00, 36.81s/it]

Simulated and logged configuration: {'cross_width': '30um', 'cross_length': '350um'}


# Checking the results!
Results get stored in `QSweeper.librarian`. It is an object named `QLibrarian`. For how to use this, I've included a separate tutorial.

In [12]:
lib = sweeper.librarian
type(lib)

QubitPresimulated.librarian.QLibrarian

In [13]:
lib.qoptions_data

,pos_x,pos_y,orientation,chip,layer,cross_width,cross_length,cross_gap,hfss_inductance,hfss_capacitance,hfss_resistance,hfss_mesh_kw_jj,q3d_inductance,q3d_capacitance,q3d_resistance,q3d_mesh_kw_jj,gds_cell_name
0,0.0um,0.0um,0.0,main,1,10um,150um,20um,10nH,0,0,0.000007,10nH,0,0,0.000007,my_other_junction
1,0.0um,0.0um,0.0,main,1,10um,350um,20um,10nH,0,0,0.000007,10nH,0,0,0.000007,my_other_junction
2,0.0um,0.0um,0.0,main,1,30um,150um,20um,10nH,0,0,0.000007,10nH,0,0,0.000007,my_other_junction
3,0.0um,0.0um,0.0,main,1,30um,350um,20um,10nH,0,0,0.000007,10nH,0,0,0.000007,my_other_junction


In [18]:
lib.simulations_data

,energy_elec,energy_elec_sub,energy_mag
0,8.211878e-25,7.536213e-25,8.055156e-27
1,4.108442e-24,3.729588e-24,8.072458e-26
2,1.155816e-24,1.061941e-24,8.520486e-27
3,5.156559e-24,4.731528e-24,7.903743e-26


As we can see, the indexes correspond from one to the other.

# Note: Autosaving Data

In these Ansys simulations, there's a high chance your run will become broken midway through. So `QSweeper` will autosave each indvidual simulation to a `.csv` file for record keeping.

When initalizing `QSweeper`, recall we input a `save_path = r'./QSweeper_Tutorial.cvs'`. Let's load that in.

**Notice**, we separate `QLibrarian.qoptions_data` and `QLibrarian.simulations_data` by a column called `__SPLITTER__`

In [45]:
import pandas as pd

df = pd.read_csv('./QSweeper_Tutorial.csv')
df

,pos_x,pos_y,orientation,chip,layer,cross_width,cross_length,cross_gap,hfss_inductance,hfss_capacitance,...,hfss_mesh_kw_jj,q3d_inductance,q3d_capacitance,q3d_resistance,q3d_mesh_kw_jj,gds_cell_name,__SPLITTER__,energy_elec,energy_elec_sub,energy_mag
0,0.0um,0.0um,0.0,main,1,10um,150um,20um,10nH,0,...,0.000007,10nH,0,0,0.000007,my_other_junction,NaN,8.211878e-25,7.536213e-25,8.055156e-27
1,0.0um,0.0um,0.0,main,1,10um,350um,20um,10nH,0,...,0.000007,10nH,0,0,0.000007,my_other_junction,NaN,4.108442e-24,3.729588e-24,8.072458e-26
2,0.0um,0.0um,0.0,main,1,30um,150um,20um,10nH,0,...,0.000007,10nH,0,0,0.000007,my_other_junction,NaN,1.155816e-24,1.061941e-24,8.520486e-27
3,0.0um,0.0um,0.0,main,1,30um,350um,20um,10nH,0,...,0.000007,10nH,0,0,0.000007,my_other_junction,NaN,5.156559e-24,4.731528e-24,7.903743e-26
